# Plan

1. Run local LLMs with Langchain
2. Simple local agents
3. Orchestrate multiple agents: CrewAI <-

In [1]:
# %pip install --upgrade crewai
# %pip install --upgrade 'crewai[tools]'

# 3. Multiple Agents with CrewAI

Using `Langchain` for multiple agents is good, with low-level control. However, it's mainly designed for OpenAI models (with tools).

We're gonna use `crewai` for multiple agents orchestration.

Our goal: market research, generate business plan for a new startup!

> Generative AI in finalcial services?

In [2]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.agents import load_tools
from langchain.agents import Tool

from langchain.llms import Ollama
from langchain_community.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from crewai import Agent, Task, Process, Crew

# load api key from .env
load_dotenv(find_dotenv())

sid = os.getenv("GOOGLE_CSE_ID")
google_api_key = os.getenv("GOOGLE_API_KEY")


In [3]:
# load GPT-4
api = os.environ.get("OPENAI_API_KEY")

# load local model 

wizard = "./models/wizardlm-13b-v1.2.Q4_0.gguf"

local_llm = GPT4All(
  model=wizard, 
  callbacks=[StreamingStdOutCallbackHandler()], 
  verbose=True
)

In [4]:
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

## define tools to use, can build custom tools

# google search
google = GoogleSearchAPIWrapper(google_api_key=google_api_key, google_cse_id=sid)
search_gg = Tool(
    name="search google",
    description="Search Google for up to date results",
    func=google.run
)


# wikipedia search
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=2000))
search_wiki = Tool(
    name="search wikipedia",
    description="Search Wikipedia articles for grounded facts and concepts",
    func=wikipedia.run
)

In [5]:
marketer = Agent(
        role="Market Research Analyst",
        goal="Gather information on the demand for a product or service, the target audience, and the competition.",
        backstory="""You're a serial entrepreneur who has been successful in the past. You're now looking to start a new business and need to do some market research to determine the demand for your product or service, the target audience, and the competition. 
        This will help you make informed decisions about your business and increase your chances of success. You want to look for ideas or products that are feasible technology-wise and have a good market potential
        """,
        verbose=True,
        memory=True,
        allow_delegation=False,
        max_rpm=10,
        max_iter=5,
        llm=local_llm # load our custom model
)

technologist = Agent(
        role="Technology Expert",
        goal="Make assessment on how technologically feasable the company is and what type of technologies the company needs to adopt in order to succeed",
        backstory="""You are an expert in AI technology, with a deep understanding of both current and emerging technological trends. Your 
                expertise lies not just in knowing the technology but in foreseeing how it can be leveraged to solve real-world problems and drive business innovation.
                You have a knack for identifying which technological solutions best fit different business models and needs, ensuring that companies stay ahead of 
                the curve.""",
        verbose=True,
        memory=True,
        allow_delegation=False, # enable collaboration between agent
        # llm=local_llm # default is openai
)

founder = Agent(
        role="Founder, ex-CEO of WeDontWork",
        goal="Evaluate and advise on the business model, scalability, and potential revenue streams to ensure long-term sustainability and profitability",
        backstory="""You are a seasoned professional with expertise in shaping business strategies. Your insight is essential for turning innovative ideas 
                into viable business models. Your experience in scalability ensures that a business can grow without compromising its values or operational efficiency. 
                Your goal is not just about immediate monetary gains but about building a resilient  business that can thrive in a changing market.""",
        verbose=True,
        allow_delegation=True,  
        # llm=local_llm
)

In [6]:
topic = "Generative AI in financial services"

market = Task(
  description=f'Find and summarize the latest potential business ideas on {topic}',
  expected_output=f'A bullet list summary of the top 5 most promising business ideas on {topic}',
  agent=marketer,
  tools=[search_gg]
)

technology = Task(
  description=f'Analyze how to build a product or service on {topic} with sustainable growth and profit.',
  expected_output = f'A detailed report in bullet points, maximum 10 entries. It has to address the most important areas to build and deploy a business on {topic}.',
  agent=technologist,
  tools=[search_wiki, search_gg]
)

business = Task(
  description=f'Analyze and summarize market fit and technological report on {topic}. Write a detailed business plan with description each steps.',
  expected_output=f'A business plan with a comprehensive analysis in 5 bullet points on {topic}, making clear the business model and growth path.',
  agent=founder,
  tools=[search_gg]
)

In [7]:
crew = Crew(
    agents=[marketer, technologist, founder],
    tasks=[market, technology, business],
    verbose=2,
    process=Process.sequential,# Sequential process will have tasks executed one after the other and the outcome of the previous one is passed as extra content into this next.
)

result = crew.kickoff()

print('***'*20 + "*****Final Result*****" + '***'*20)
print(result)

 [DEBUG]: == Working Agent: Market Research Analyst
 [INFO]: == Starting Task: Find and summarize the latest potential business ideas on Generative AI in financial services


> Entering new CrewAgentExecutor chain...
Action: search google
Action Input: {"query": "Generative AI in financial services"}Action: search google
Action Input: {"query": "Generative AI in financial services"} 

Oct 3, 2023 ... Gen AI can help bank employees effectively find and understand information in contracts (e.g., policies, credit memos, underwriting, trading, ... Dec 5, 2023 ... The strategic deployment of tailored gen AI solutions enables financial institutions to profoundly enhance their service operations and improve ... Nov 26, 2023 ... According to a McKinsey report, generative AI could add $2.6 trillion to $4.4 trillion annually in value to the global economy. The banking ... Apr 19, 2023 ... The advent of generative AI is a dramatic platform change for financial services companies with the potentia

In [9]:
# export result to markdown
with open('result.md', 'w') as f:
  f.write(f'This is an example result of coordinating multiple AI agents on {topic} \n')
  f.write('======\n')
  f.write(result)
  f.close()